In [25]:
import ee
import geemap
import matplotlib.pyplot as plt
import numpy as np

In [26]:
Map = geemap.Map()

In [27]:
time_start = '2022-01-01'
time_end = '2023-01-01'

In [28]:
# test area
geometry = ee.Geometry.Polygon([
    [[68.06523955643267,27.402689139647947],
     [68.68802703201861,27.402689139647947],
     [68.68802703201861,27.925089418721566],
     [68.06523955643267,27.925089418721566],
     [68.06523955643267,27.402689139647947]]
])

In [29]:
Map.centerObject(geometry, 8)

In [30]:
# function to reduce speckle noise
def despeckel(img):
    return img.focal_mean(30, 'square', 'meters').copyProperties(img, img.propertyNames())

In [31]:
image_collection = ee.ImageCollection("COPERNICUS/S1_GRD")

In [32]:
before = (image_collection
    .filterDate(time_start, time_end)
    .filter(ee.Filter.calendarRange(3, 3, 'month'))
    .filterBounds(geometry)
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .select('VV')
    .map(despeckel)
    .min())

# adding the before layer to the map with 90% stretch
before_min = before.reduceRegion(
    reducer=ee.Reducer.percentile([10]),
    geometry=geometry,
    scale=30
).get('VV')

before_max = before.reduceRegion(
    reducer=ee.Reducer.percentile([90]),
    geometry=geometry,
    scale=30
).get('VV')

Map.addLayer(before.clip(geometry), 
              {'min': before_min, 'max': before_max}, 
              'Before', False)

In [33]:
after = (image_collection
    .filterDate(time_start, time_end)
    .filter(ee.Filter.calendarRange(8, 8, 'month'))
    .filterBounds(geometry)
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .select('VV')
    .map(despeckel)
    .min())

# add theafter layer to the map with 90% stretch
after_min = after.reduceRegion(
    reducer=ee.Reducer.percentile([10]),
    geometry=geometry,
    scale=30
).get('VV')

after_max = after.reduceRegion(
    reducer=ee.Reducer.percentile([90]),
    geometry=geometry,
    scale=30
).get('VV')

Map.addLayer(after.clip(geometry), 
              {'min': after_min, 'max': after_max}, 
              'After', False)


In [34]:
# calculate change detection
change = before.subtract(after)

# adiding the change layer to the map with 90% stretch
change_min = change.reduceRegion(
    reducer=ee.Reducer.percentile([10]),
    geometry=geometry,
    scale=30
).get('VV')

change_max = change.reduceRegion(
    reducer=ee.Reducer.percentile([90]),
    geometry=geometry,
    scale=30
).get('VV')

Map.addLayer(change.clip(geometry), 
              {'min': change_min, 'max': change_max}, 
              'Flooded Region', False)

In [35]:
# change values for the histogram
change_data = change.reduceRegion(
    reducer=ee.Reducer.toList(),
    geometry=geometry,
    scale=30,
    maxPixels=1e9
).getInfo()

# extract the list of values
change_values = change_data['VV']

# plot the histogram using matplotlib
plt.figure(figsize=(10, 6))
plt.hist(change_values, bins=30, color='blue', edgecolor='black')
plt.title('Histogram of Change Detection')
plt.xlabel('Change Value')
plt.ylabel('Frequency')
plt.show()

In [36]:
threshold = 6.5

In [37]:
# add thresholded flood layers to the map
Map.addLayer(change.gt(threshold).clip(geometry), {}, 'Flood Threshold', False)
Map.addLayer(before.gt(-16).clip(geometry), {}, 'Water Before', False)
Map.addLayer(after.gt(-16).clip(geometry), {}, 'Water After', False)

In [39]:
# display the map
Map

Map(bottom=373385.4146102466, center=[45.91833876927136, -0.10775146755974022], controls=(WidgetControl(option…

In [15]:
folder_name = 'GEE_Exports'

In [113]:
export_task = ee.batch.Export.image.toDrive(
    image=before.clip(geometry), 
    description='before_flood_tif_export',
    scale=30,
    region=geometry, 
    fileFormat='GeoTIFF', 
    folder=folder_name
)

export_task.start()
print(f"Exporting GeoTIFF to Google Drive folder: {folder_name}...")

Exporting GeoTIFF to Google Drive folder: GEE_Exports...


In [114]:
export_task = ee.batch.Export.image.toDrive(
    image=after.clip(geometry), 
    description='after_flood_tif_export',
    scale=30,
    region=geometry, 
    fileFormat='GeoTIFF', 
    folder=folder_name
)

export_task.start()
print(f"Exporting GeoTIFF to Google Drive folder: {folder_name}...")

Exporting GeoTIFF to Google Drive folder: GEE_Exports...
